In [ ]:
#download da biblioteca pycatch22 para extracao dos 22 atributos do dataset
!pip install pycatch22
import pycatch22

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# importacoes
import os
import csv

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import zipfile
import threading
from scipy import signal

root = "C:\\Users\\radek\\Desktop\\PUC\\tcc\\"
directory = root+"Rapid_age-grading_and_species_identification_of_natural_mosquitoes\\"
# directory = root+"1. IR Spectra of Mosquitoes/"
files_directory = os.listdir(directory)
dataset = pd.DataFrame()
dataset_resampled = pd.DataFrame()

In [ ]:
# funcao para normalizacao dos valores de absorcao
def znorm(x):
  x_znorm = (x - np.mean(x)) / np.std(x)
  return x_znorm

#funcao para atribuir o valor mais proximo aos faltantes
def closest_index(grouped, index):
    left = grouped.index[grouped.index < index].max()
    right = grouped.index[grouped.index > index].min()
    if pd.isna(left):
        # print(f"Mais proximo a direita: {right}")
        return grouped[right]
    elif pd.isna(right):
        # print(f"Mais proximo a esquerda: {left}")
        return grouped[left]
    else:
        # print(f"Inserido a media de {left} e {right} para {index}")
        return grouped[left] + grouped[right] / 2

#funcao para gerar o dataset catch22 atraves do dataset_resampled
def get_catch22(dataset_resampled):
  df = dataset_resampled.drop(["species", "age"], axis=1)
  dta_aux = []

  for s in df.index:
    series = df.iloc[s]
    s = dataset_resampled.iloc[s]
    species = s["species"]
    age = s["age"]
    values = pycatch22.catch22_all(series)["values"] + [age, species]
    dta_aux.append(values)

  colunas = [1000 + i for i in range(22)] + ["age", "species"]
  dataset_22 = pd.DataFrame(dta_aux, columns=colunas)

  for col in colunas:
    df[str(col)] = dataset_22[col]

  return df



In [ ]:
def extrair_mzz_files():
    for root, dirs, files in os.walk(directory):
        for name in files:
            if name.endswith('.mzz'):
                caminho_arquivo_mzz = os.path.join(root, name)
                with zipfile.ZipFile(caminho_arquivo_mzz, 'r') as zip_ref:
                    zip_ref.extractall(root)

                pasta_extraida = os.path.splitext(caminho_arquivo_mzz)[0]
                os.remove(caminho_arquivo_mzz)


In [ ]:
def find_age(splitted):
    index = next((i for i, item in enumerate(splitted) if 'D' in item), -1)

    if index != -1:
        return int(splitted[index][:-1])
    return int(splitted[1])

def find_specie(splitted):
    if splitted[0] == 'AA':
        return 'AR'
    elif splitted[0] == 'AR':
        return 'AR'
    elif splitted[0] == 'AG':
        return 'KS'
    elif splitted[0] == 'KS':
        return 'KS'
    return splitted[0]


In [ ]:
def extrair_mzz_files():
  for root, dirs, files in os.walk(directory):
      for name in files:
          if name.endswith('.mzz'):
              caminho_arquivo_mzz = os.path.join(root, name)

              with zipfile.ZipFile(caminho_arquivo_mzz, 'r') as zip_ref:
                  zip_ref.extractall(root)

              pasta_extraida = os.path.splitext(caminho_arquivo_mzz)[0]
              print(caminho_arquivo_mzz)

              os.remove(pasta_extraida)

#funcao para concatenar os datasets separados pelas threads
def concat_datasets(folders):
    features = []
    resampleds = []
    for folder in folders:
        df_features = pd.read_csv(directory+'features_originais-'+folder+'.csv', sep=';')
        df_resampled = pd.read_csv(directory+'dataset_resampled-'+folder+'.csv', sep=';')
        features.append(df_features)
        resampleds.append(df_resampled)

    print('- Generating database features_originais')
    concatenated_features = pd.concat([features[0], features[1], features[2]], ignore_index=True)
    print('- Generating database dataset_resampled')
    concatenated_resampled = pd.concat([resampleds[0], resampleds[1], resampleds[2]], ignore_index=True)
    print('- Generating database dataset_catch22')
    dataset_22 = get_catch22(concatenated_resampled)

    print('- Saving dataset features_originais')
    concatenated_features.to_csv(directory+'features_originais.csv', index=False, sep=';')
    print('- Saving dataset dataset_resampled')
    concatenated_resampled.to_csv(directory+'dataset_resampled.csv', index=False, sep=';')
    print('- Saving dataset dataset_catch22')
    dataset_22.to_csv(directory+'dataset_22.csv', index=False, sep=';')

def calculate_wavenumber(a,b,c):
  wvn_n = []
  for i in range(0, int(c)):
    wvn_i = a - (i * (a - b) / c )
    wvn_n.append(wvn_i)
  return wvn_n


In [ ]:
def parte(foldername):
    dataset = pd.DataFrame()
    dataset_resampled = pd.DataFrame()
    wavenumbers = [3856, 3400, 3275, 2923, 2859, 1901, 1746, 1636, 1539, 1457, 1307, 1154, 1076, 1027, 880, 526, 401]
    for root_tmp, dirs_tmp, files_tmp in os.walk(directory):
      if foldername in root_tmp:
          print(f'\n ------------------------------------------{foldername}: \n{root_tmp}\n------------------------------------------------')
          for file_tmp in files_tmp:
            if file_tmp.endswith('.tmp'):
              hasAge = find_age(file_tmp.split('-'))
              if file_tmp.split('-')[2] != 'UNK' and file_tmp.split('-')[0] != 'AC' and file_tmp.split('-')[0] != 'ACB' and file_tmp.split('-')[2] != '99D' and hasAge != -1:
                caminho_arquivo_tmp = os.path.join(root_tmp, file_tmp)

                df = pd.read_csv(caminho_arquivo_tmp, sep='\s+', header=None)
                df.columns = ['absorbance']

                a = df['absorbance'].iloc[0]
                b = df['absorbance'].iloc[1]
                c = df['absorbance'].iloc[2]

                df_absorbance = df.iloc[3:].copy()
                df_absorbance['wavenumber'] = calculate_wavenumber(a,b,c)

                #dataset que sera gerado com 1000 colunas mas com todos atributos referentes ao wavenumber
                data_signal = pd.Series(znorm(signal.resample(df_absorbance['absorbance'], 1000))) #downsampling do sinal de 15000 ou 1700 para 1000 colunas
                data_signal['species'] = find_specie(file_tmp.split('-')) # adiciona a espécie na penúltima coluna
                data_signal['age'] = find_age(file_tmp.split('-')) # adiciona a idade (int) na última coluna
                data_resampled = pd.DataFrame(data_signal).transpose() # adiciona serie temporal com normalizacao ao dataframe data_resampled
                dataset_resampled = dataset_resampled.append(data_resampled, ignore_index=True) # anexa o conjunto referente a esse arquivo ao dataset final

                df_absorbance['wavenumber'] = df_absorbance['wavenumber'].astype(int) # realiza conversão dos valores referentes ao wavenumber para inteiro
                grouped = df_absorbance.groupby('wavenumber')['absorbance'].mean() # formata em grupos os wavenumbers iguais e realiza media para cada grupo
                #checagem para verificar se os valores de absorbance referentes aos wavenumbers selecionados estao no dataset
                for index in wavenumbers:
                  #caso nao esteja atribui o valor mais proximo para absorbance referente aquele wavenumber
                  if not grouped.index.isin([index]).any():
                    new_absorbance = closest_index(grouped, index)
                    grouped.loc[index] = new_absorbance

                grouped = grouped.sort_index()
                grouped = grouped[grouped.index.isin(wavenumbers)]
                #separa os valores do wavenumber como header do dataset e absorbance como values normalizados.
                dataset_key_value = pd.DataFrame({'header': grouped.index, 'value': znorm(grouped.values)})
                dataset_key_value.set_index('header', inplace=True)
                dataset_key_value = dataset_key_value.T
                dataset_key_value['species'] = find_specie(file_tmp.split('-'))
                dataset_key_value['age'] = find_age(file_tmp.split('-'))
                dataset = dataset.append(dataset_key_value, ignore_index=True)

    #transforma o dataset padrão com 17 atributos em arquivo csv
    dataset['age'] = pd.cut(dataset['age'], bins=[1, 5, 11, 18], labels=['1-4', '5-10', '11-17'], right=False)
    dataset.to_csv(directory+'features_originais-'+foldername+'.csv', index=False, sep=';')

    #transforma o dataset que foi feito resampled com todos atributos em arquivo csv
    dataset_resampled['age'] = pd.cut(dataset_resampled['age'], bins=[1, 5, 11, 18], labels=['1-4', '5-10', '11-17'], right=False)
    dataset_resampled.to_csv(directory+'dataset_resampled-'+foldername+'.csv', index=False, sep=';')

    #cria e gera um dataset utilizando pycatch22 para extrair 22 atributos
    # dataset_22 = get_catch22(dataset_resampled)
    # dataset_22.to_csv(directory+'dataset_22-'+foldername+'.csv', index=False, sep=';')

In [ ]:
folders = ["1. Laboratory Variation", "2. Genetic Variation", "3. Environmental Variation"]

In [ ]:
thread1 = threading.Thread(target=parte, args=(folders[0],))
thread2 = threading.Thread(target=parte, args=(folders[1],))
thread3 = threading.Thread(target=parte, args=(folders[2],))

thread1.start()
thread2.start()
thread3.start()

thread1.join()
thread2.join()
thread3.join()

In [ ]:
#concatena os datasets para um unico dataset
concat_datasets(folders)

In [ ]:
data_features = pd.read_csv(directory+'features_originais.csv', sep=';')

print(data_features.groupby(['age']).size())

In [ ]:
!pip install xgboost

In [ ]:
import joblib
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import  model_selection
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn import preprocessing
import pandas as pd
parameters_KNN = {
    'n_neighbors': [1, 3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree'],
    'leaf_size': [5, 10, 20, 30],
    'p': [1, 2],
    'metric': ['euclidean', 'manhattan']
  }


parameters_LR = {
    'penalty': [None,'l1', 'l2', 'elasticnet'],
    'C': [0.1, 1.0, 5.0],
    'solver': ['liblinear', 'lbfgs'],
    'max_iter': [100, 200, 300, 500],
    'class_weight': [None, 'balanced']
}

parameters_SVC = {
    'C': [0.1, 0.5, 1, 3, 5],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto'],
    'class_weight': [None, 'balanced']
}
parameters_RF = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 3, 5, 7],
    'min_samples_split': [2, 5, 7, 10],
    'criterion': ['gini', 'entropy']
}
parameters_XGB = {
    'max_depth': [1, 3, 5, 7],
    'learning_rate': [0.1, 0.01],
    'n_estimators': [100, 200, 300],
    'gamma': [0, 0.1],
    'min_child_weight': [1, 3, 5]
}

def classificar(name_file, file, X, y) -> pd.DataFrame:

    # separando uma parte para base de validação (30%)
    X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.7, random_state=42, stratify=y)

    clfKNN = KNeighborsClassifier()
    clfLR = LogisticRegression()
    clfSVC = SVC()
    clfRF = RandomForestClassifier()
    clfXGB = XGBClassifier()

    folds = 10

    # Define as configurações para o GridSearchCV
    grid_searches = [
        (clfKNN, parameters_KNN),
        (clfLR, parameters_LR),
        (clfSVC, parameters_SVC),
        (clfRF, parameters_RF),
        (clfXGB, parameters_XGB)
    ]

    def run_grid_search(clf, params):
        gs = GridSearchCV(clf, params, scoring='accuracy', cv=folds, n_jobs=-1)
        gs.fit(X_train, y_train)
        return gs

    # Executa os GridSearch em paralelo
    results = joblib.Parallel(n_jobs=-1)(
        joblib.delayed(run_grid_search)(clf, params) for clf, params in grid_searches
    )

    # Extrai os resultados de cada classificador
    df_results = []
    best_estimators = []
    for i, (clf, _) in enumerate(grid_searches):
        gs = results[i]
        df_results.append(gs.cv_results_)
        best_estimators.append(gs.best_estimator_)

        print(f"Best params for {clf.__class__.__name__}: ")
        print(gs.best_params_)

    # Utiliza os melhores estimadores para fazer as previsões
    y_preds = []
    accuracies = []
    matriz_confusoes = []
    for clf, y_pred_name in zip(best_estimators, ['KNN', 'LR', 'SVC', 'RF', 'XGB']):
        y_pred = clf.predict(X_val)
        accuracy = accuracy_score(y_val, y_pred)
        matriz_confusao = confusion_matrix(y_val, y_pred)

        print(f"\n{y_pred_name}: \n")
        print("Acuracia: {:.5f}".format(accuracy))
        print("Matriz de Confusão - {}: ".format(y_pred_name))
        print(matriz_confusao)

        y_preds.append(y_pred)
        accuracies.append(accuracy)
        matriz_confusoes.append(matriz_confusao)

    resultados = {
        'Classificador': ['KNN', 'Logistic Regression', 'SVC', 'Random Forest', 'XGBoost'],
        'Melhores Parâmetros': [gs.best_params_ for gs in results],
        'Melhor Acurácia': accuracies
    }

    df_resultados = pd.DataFrame(resultados)

    df_resultados.to_csv(root + f'resultados_classificadores_{file}.csv', sep=';', index=False)

    return df_resultados


In [ ]:
files = ['features_originais']
dfs = []

for file in files:
    name_file = directory + f"{file}.csv"

    data_age = pd.read_csv(name_file, sep=';')
    data_age['age'] = data_age['age'].map({'1-4': 0, '5-10': 1, '11-17': 2})
    print(f"\n\n\n\nLendo dataset {name_file}........")

    X = data_age.drop([ 'age', 'species'], axis=1)
    y = data_age['age']
    df_gerado = classificar(name_file, file, X, y)

    df_gerado["dataset"] = file
    dfs.append(df_gerado)

merged_dataset = pd.concat(dfs, axis=1)
merged_dataset.to_csv(directory + f'resultados_classificadores_idades.csv')